In [6]:
from Trung_Chien_datamodel import TradingData
from Trung_Chien_MomentumHSI_posgen import GeneratePosition
import telegram
import asyncio

trading_data = TradingData(
    symbol='vn30f1m',
    host='192.168.1.35',
    port='5432',
    dbname='postgres',
    user='postgres',
    password='postgres',
    schema='public',
    csv_file='vn30f1m_paper_trade.csv'
)

pos_gen = GeneratePosition()
bot = telegram.Bot(token='7117934389:AAFvk_lHb-kKg8Z75z2gZ4NwSjIleetHjdQ')

async def main():
    df = trading_data.get_vn30f1m()
    df = trading_data.get_vn30f1m_trading(df)
    trading_data.update_data(df)
    
    feature = pos_gen.get_feature()
    position = pos_gen.posgen(feature)
    
    if position is not None:
        updated_df = trading_data.update_position(position)
        message = updated_df[['Date', 'Close', 'Position']].tail(1).to_string(index=False)
        bot.send_message(chat_id='-4270845760', text=f'<pre>{message}</pre>', parse_mode='HTML')
        print(updated_df)
    else:
        print("Failed to update the position")

await main()

                 Date    Open    High     Low   Close  Volume  basis  Position
0 2024-07-10 14:00:00  1316.4  1317.0  1316.2  1316.8   631.0   0.54        -1
